# Example 6: Cloud Logging - Datadog Integration

## What You'll Learn

This notebook demonstrates cloud logging with Datadog:

- **DatadogHandler** - Send logs to Datadog
- **Datadog_ServiceConfig** - Configure Datadog integration
- **API Key Management** - Secure configuration
- **Cloud vs Local** - When to use cloud logging
- **Real-World Examples** - Production monitoring scenarios


## Setup - Imports


In [3]:
import asyncio
import os
from dc_logger.client.Log import LogEntry, LogLevel
from dc_logger.client.base import Logger, HandlerInstance
from dc_logger.services.cloud.datadog import DatadogHandler, Datadog_ServiceConfig

print("All imports successful!")


All imports successful!


---
## Part 1: Basic Datadog Configuration

**Note**: To actually send logs to Datadog, you need:
1. A Datadog account
2. A valid API key
3. Optionally, an APP key for additional features

For this example, we'll show the configuration. Replace placeholders with your actual keys.


In [4]:
print("=" * 70)
print("Basic Datadog Configuration")
print("=" * 70)

# Example configuration - USE ACTUAL KEYS FOR REAL USAGE
datadog_config = Datadog_ServiceConfig(
    api_key=os.getenv("DATADOG_API_KEY", "YOUR_API_KEY_HERE"),  # Get from environment
    app_key=os.getenv("DATADOG_APP_KEY", "YOUR_APP_KEY_HERE"),  # Optional
    site="datadoghq.com",  # or us3.datadoghq.com, us5.datadoghq.com, etc.
    service="my_application",  # Your service name in Datadog
    env="production"  # Environment: production, staging, development
)

print("\nDatadog Configuration:")
print(f"  Site: {datadog_config.site}")
print(f"  Service: {datadog_config.service}")
print(f"  Environment: {datadog_config.env}")
print(f"  Intake URL: {datadog_config.derive_intake_url()}")
print(f"  API Key: {'Set' if datadog_config.api_key != 'YOUR_API_KEY_HERE' else 'Not Set'}")


Basic Datadog Configuration

Datadog Configuration:
  Site: datadoghq.com
  Service: my_application
  Environment: production
  Intake URL: https://http-intake.logs.datadoghq.com/v1/input
  API Key: Not Set


---
## Part 2: Creating Datadog Logger

**Important**: The following cells demonstrate the setup. If you don't have valid Datadog credentials, the actual log sending will fail (which is expected).


In [5]:
print("=" * 70)
print("Creating Datadog Logger")
print("=" * 70)

# Note: This will work if you have valid credentials
# Otherwise, it will show configuration but not send logs

try:
    # Create Datadog handler
    datadog_handler = DatadogHandler(config=datadog_config)
    
    # Create handler instance
    datadog_handler_instance = HandlerInstance(
        handler=datadog_handler,
        output_type="json"  # Datadog accepts JSON format
    )
    
    # Create logger with Datadog handler
    datadog_logger = Logger(
        app_name="cloud_demo",
        handlers=[datadog_handler_instance]
    )
    
    print("\nDatadog logger created successfully!")
    print("Ready to send logs to Datadog")
    
except Exception as e:
    print(f"\nNote: Could not initialize Datadog handler: {e}")
    print("This is expected if you don't have valid Datadog credentials")
    print("Configuration shown above is for reference")


Creating Datadog Logger

Note: Could not initialize Datadog handler: Can't instantiate abstract class DatadogHandler without an implementation for abstract method 'flush'
This is expected if you don't have valid Datadog credentials
Configuration shown above is for reference


---
## Part 3: Example Log Entries for Datadog

Here's what log entries would look like when sent to Datadog.


In [6]:
print("=" * 70)
print("Example Log Entries for Datadog")
print("=" * 70)

# These examples show what you would log if you had valid credentials

examples = [
    {
        "level": "INFO",
        "message": "Application started",
        "kwargs": {
            "extra": {"version": "1.2.3", "environment": "production"}
        }
    },
    {
        "level": "INFO",
        "message": "User authenticated",
        "kwargs": {
            "user_id": "user_alice_123",
            "tenant_id": "tenant_acme",
            "action": "login",
            "extra": {"login_method": "oauth2"}
        }
    },
    {
        "level": "WARNING",
        "message": "High memory usage",
        "kwargs": {
            "extra": {"memory_percent": 85, "threshold": 80}
        }
    },
    {
        "level": "ERROR",
        "message": "Database connection failed",
        "kwargs": {
            "extra": {
                "host": "db.example.com",
                "error": "Connection timeout",
                "retry_count": 3
            }
        }
    }
]

print("\nExample logs that would be sent to Datadog:\n")
for i, ex in enumerate(examples, 1):
    print(f"{i}. [{ex['level']}] {ex['message']}")
    if 'user_id' in ex['kwargs']:
        print(f"   User: {ex['kwargs']['user_id']}")
    if 'action' in ex['kwargs']:
        print(f"   Action: {ex['kwargs']['action']}")
    if 'extra' in ex['kwargs']:
        print(f"   Extra: {ex['kwargs']['extra']}")
    print()


Example Log Entries for Datadog

Example logs that would be sent to Datadog:

1. [INFO] Application started
   Extra: {'version': '1.2.3', 'environment': 'production'}

2. [INFO] User authenticated
   User: user_alice_123
   Action: login
   Extra: {'login_method': 'oauth2'}

3. [WARNING] High memory usage
   Extra: {'memory_percent': 85, 'threshold': 80}

4. [ERROR] Database connection failed
   Extra: {'host': 'db.example.com', 'error': 'Connection timeout', 'retry_count': 3}



---
## Part 4: Different Datadog Sites

Datadog has different sites depending on your region.


In [7]:
print("=" * 70)
print("Datadog Sites and Intake URLs")
print("=" * 70)

sites = [
    "datadoghq.com",  # US1
    "us3.datadoghq.com",  # US3
    "us5.datadoghq.com",  # US5
    "datadoghq.eu",  # EU
    "ap1.datadoghq.com",  # AP1
]

print("\nDatadog site intake URLs:\n")
for site in sites:
    url = Datadog_ServiceConfig._derive_intake_url(site)
    print(f"  {site:25} -> {url}")

print("\nNote: Use the site that matches your Datadog account region")


Datadog Sites and Intake URLs

Datadog site intake URLs:

  datadoghq.com             -> https://http-intake.logs.datadoghq.com/v1/input
  us3.datadoghq.com         -> https://http-intake.logs.us3.datadoghq.com/v1/input
  us5.datadoghq.com         -> https://http-intake.logs.us5.datadoghq.com/v1/input
  datadoghq.eu              -> https://http-intake.logs.datadoghq.eu/v1/input
  ap1.datadoghq.com         -> https://http-intake.logs.ap1.datadoghq.com/v1/input

Note: Use the site that matches your Datadog account region


---
## Summary

1. **DatadogHandler** - Send logs to Datadog cloud service
2. **Datadog_ServiceConfig** - Configure Datadog integration with API keys
3. **Different Sites** - Support for all Datadog regions
4. **Secure Configuration** - Use environment variables for API keys
5. **Best Practices** - Security, cost optimization, monitoring
6. **Use Cases** - When to use cloud vs file logging

### Key Takeaways

- **Centralized Monitoring** - All logs in one place
- **Real-Time Alerts** - Get notified of issues immediately
- **Powerful Search** - Query across all services
- **Cost Awareness** - Monitor usage to control costs
- **Security First** - Never expose API keys

### Configuration Checklist

Before using Datadog logging in production:

- [ ] Get Datadog API key from your Datadog account
- [ ] Store API key in environment variable (not in code)
- [ ] Verify correct site for your region
- [ ] Set appropriate service and environment names
- [ ] Test with a few logs first
- [ ] Set up alerts for log ingestion failures
- [ ] Configure log retention policies

